In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab7").getOrCreate()

24/09/16 10:39:48 WARN Utils: Your hostname, mahe-HP-Z1-G9-Tower-Desktop-PC resolves to a loopback address: 127.0.1.1; using 172.16.57.86 instead (on interface eno1)
24/09/16 10:39:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/16 10:39:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.option("inferSchema", True).option("header", False).csv("kddcup.data_10_percent_corrected")

In [3]:
column_names = [ "duration", "protocol_type", "service", "flag","src_bytes", "dst_bytes", "land", "wrong_fragment", 
                "urgent","hot", "num_failed_logins", "logged_in", "num_compromised","root_shell", "su_attempted", 
                "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", 
                "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", 
                "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", 
                "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", 
                "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", 
                "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label"]

data = data.toDF(*column_names)
data.head()

24/09/16 10:39:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(duration=0, protocol_type='tcp', service='http', flag='SF', src_bytes=181, dst_bytes=5450, land=0, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, logged_in=1, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmds=0, is_host_login=0, is_guest_login=0, count=8, srv_count=8, serror_rate=0.0, srv_serror_rate=0.0, rerror_rate=0.0, srv_rerror_rate=0.0, same_srv_rate=1.0, diff_srv_rate=0.0, srv_diff_host_rate=0.0, dst_host_count=9, dst_host_srv_count=9, dst_host_same_srv_rate=1.0, dst_host_diff_srv_rate=0.0, dst_host_same_src_port_rate=0.11, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=0.0, dst_host_srv_serror_rate=0.0, dst_host_rerror_rate=0.0, dst_host_srv_rerror_rate=0.0, label='normal.')

In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler

def kmeans(data, k):

    assembler = VectorAssembler().setInputCols(numeric.columns[:-1]).setOutputCol("featureVector")
    
    scaler = StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector") \
    .setWithStd(True).setWithMean(False)

    kmeans = KMeans().setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster") \
    .setFeaturesCol("scaledFeatureVector")

    pipeline = Pipeline().setStages([assembler, scaler, kmeans])

    model = pipeline.fit(numeric)

    kmeans_model = model.stages[-1]

    training_cost = kmeans_model.summary.trainingCost

    return training_cost

In [10]:
numeric = data.drop("protocol_type", "service", "flag").cache()

for k in list(range(20,101,20)):
    print(k, kmeans(numeric, k))

24/09/16 10:40:28 WARN CacheManager: Asked to cache already cached data.
24/09/16 10:40:32 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


20 3292760.0271574617


40 978756.2232713241


60 554460.9308034047
80 385098.24050851556


100 292475.13841435744
